## Ortam Değişkenlerini Yükleyin
### Ortam değişkenlerini .env dosyasından yüklemek için dotenv kütüphanesini kullanacağız:

In [7]:
# hücre 1

import os
from dotenv import load_dotenv

# .env dosyasını yükle
load_dotenv()

# OpenAI API anahtarını al
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Veriyi Yükleyin ve İşleyin

### Veri dosyalarını (txt) okuyup, her bir dosyanın içeriğini vektör haline getirmek için OpenAI Embeddings’i kullanacağız. Ardından, bu vektörleri ChromaDB’ye ekleyeceğiz.

In [8]:
# Hücre 2: Veriyi Yükleyin ve Ayıklayın

import os
import re

data_directory = "rag_data/website/organized_data"

def extract_qa_pairs(text):
    """
    Bir txt dosyasındaki tüm soru-cevap çiftlerini çıkartır.
    """
    qa_pairs = []
    pattern = r"(\d+)\.\s*Question:(.*?)\n\s*Answer:(.*?)(?=\n\d+\.|\Z)"
    matches = re.findall(pattern, text, re.DOTALL)
    
    for match in matches:
        question = match[1].strip()
        answer = match[2].strip()
        qa_pairs.append({"question": question, "answer": answer})
    
    return qa_pairs

def load_data(directory):
    """
    Klasördeki tüm txt dosyalarını yükler ve bunları kategorilere göre sınıflandırır.
    """
    categorized_data = {}
    
    for category in os.listdir(directory):
        category_path = os.path.join(directory, category)
        if os.path.isdir(category_path):
            categorized_data[category] = []
            
            for file_name in os.listdir(category_path):
                if file_name.endswith(".txt"):
                    file_path = os.path.join(category_path, file_name)
                    with open(file_path, "r", encoding="utf-8") as f:
                        content = f.read()
                        qa_pairs = extract_qa_pairs(content)
                        categorized_data[category].extend(qa_pairs)
    
    return categorized_data

categorized_data = load_data(data_directory)
print(f"Yüklendi {sum(len(v) for v in categorized_data.values())} adet soru-cevap çifti.")

Yüklendi 1923 adet soru-cevap çifti.


## Embeddings ve Veritabanına Yükleme

### Belge başına soru-cevap çiftlerini kullanarak vektörleri oluşturup ChromaDB’ye ekleyeceğiz. Kategorilere göre verileri ayrı koleksiyonlara ayıracağız.

In [15]:
# Hücre3: Embeddings ve Veritabanına Yükleme

import unicodedata
from langchain.embeddings import OpenAIEmbeddings
from chromadb import Client
from chromadb.config import Settings

# OpenAI Embeddings başlat
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# ChromaDB Client oluştur
chroma_client = Client(Settings())

def normalize_text(text):
    """
    Özel karakterleri kaldırarak metni normalleştirir.
    """
    # Özel karakterleri değiştir
    replacements = {
        'ö': 'o', 'ü': 'u', 'ä': 'a', 'ß': 'ss', 
        'ğ': 'g', 'ç': 'c', 'ş': 's', 'ı': 'i'
    }
    for search, replace in replacements.items():
        text = text.replace(search, replace)
    # Unicode normalizasyonu yap ve sadece ASCII karakterleri koru
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')

def create_embeddings(text):
    """
    Bir metni vektöre dönüştürür.
    """
    return embeddings.embed_query(text)

def valid_collection_name(name):
    """
    Koleksiyon ismini geçerli bir formata dönüştürür.
    """
    name = normalize_text(name.lower())
    # Geçerli karakterler: alfanümerik, alt çizgi ve kısa çizgi
    name = name.replace(" ", "_").replace("&", "and").replace(",", "")
    if len(name) < 3:
        name = name + "_collection"  # Kısa isimleri geçerli uzunlukta hale getir
    return name[:63]  # En fazla 63 karakter uzunluğunda olmalı

def create_and_populate_database(categorized_data):
    """
    Kategorilere göre veritabanına vektörleri ekler.
    """
    for category, qa_pairs in categorized_data.items():
        valid_name = valid_collection_name(category)

        # Eğer koleksiyon varsa sil
        try:
            chroma_client.delete_collection(name=valid_name)
            print(f"Mevcut koleksiyon '{valid_name}' silindi.")
        except:
            print(f"Koleksiyon '{valid_name}' bulunamadı, oluşturulacak.")

        # Yeni koleksiyon oluştur
        collection = chroma_client.create_collection(name=valid_name)

        for idx, qa_pair in enumerate(qa_pairs):
            question = qa_pair["question"]
            answer = qa_pair["answer"]
            vector = create_embeddings(question)  # Sadece soruyu embed edelim
            metadata = {"question": question, "answer": answer}

            # Benzersiz bir ID oluştur
            unique_id = f"{valid_name}_{idx}"

            # Koleksiyona veriyi ekle
            collection.add(ids=[unique_id], documents=[question], embeddings=[vector], metadatas=[metadata])

        print(f"{len(qa_pairs)} adet soru-cevap çifti '{valid_name}' kategorisine eklendi.")

create_and_populate_database(categorized_data)

Mevcut koleksiyon 'magentaeins' silindi.
14 adet soru-cevap çifti 'magentaeins' kategorisine eklendi.
Mevcut koleksiyon 'hilfe_bei_storungen' silindi.
6 adet soru-cevap çifti 'hilfe_bei_storungen' kategorisine eklendi.
Mevcut koleksiyon 'gerate_and_zubehor' silindi.
519 adet soru-cevap çifti 'gerate_and_zubehor' kategorisine eklendi.
Mevcut koleksiyon 'others' silindi.
750 adet soru-cevap çifti 'others' kategorisine eklendi.
Koleksiyon 'tv_collection' bulunamadı, oluşturulacak.
80 adet soru-cevap çifti 'tv_collection' kategorisine eklendi.
Koleksiyon 'vertrag_and_rechnung' bulunamadı, oluşturulacak.
103 adet soru-cevap çifti 'vertrag_and_rechnung' kategorisine eklendi.
Koleksiyon 'internet_and_telefonie' bulunamadı, oluşturulacak.
106 adet soru-cevap çifti 'internet_and_telefonie' kategorisine eklendi.
Koleksiyon 'apps_and_dienste' bulunamadı, oluşturulacak.
186 adet soru-cevap çifti 'apps_and_dienste' kategorisine eklendi.
Koleksiyon 'mobilfunk' bulunamadı, oluşturulacak.
159 adet sor

## Backend API’yi Oluşturun

### Backend API’sini oluşturmak için Langchain ve ChromaDB kullanacağız. Kullanıcıdan gelen sorulara cevap vereceğiz.

In [17]:
# Hücre 4: Backend API Oluşturma

from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate

# OpenAI Embeddings başlat
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Chroma ile bağlantı kur
def initialize_qa_chain(category):
    """
    Belirli bir kategori için RAG Retrieval QA zincirini başlatır.
    """
    # Vektör mağazasını oluştur
    collection_name = valid_collection_name(category)
    vectorstore = Chroma(collection_name, embedding_function=embeddings.embed_query)

    # OpenAI modelini başlat
    llm = OpenAI(model="gpt-4", openai_api_key=OPENAI_API_KEY)

    # Prompt şablonunu oluştur
    prompt_template = PromptTemplate(
        template="Soru: {question}\n\nCevap:",
        input_variables=["question"]
    )

    # RetrievalQA zincirini başlat
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # Burada `stuff` seçeneğini kullanıyoruz; ihtiyaca göre değiştirilebilir
        retriever=vectorstore.as_retriever(),
        prompt=prompt_template
    )
    
    return qa_chain

## Streamlit ile Frontend Oluşturun

### Kullanıcıların belirli bir kategori seçerek soru sorabilmesi için bir frontend oluşturacağız.

In [18]:
# Hücre 5: Streamlit ile API'yi Yayınlama

import streamlit as st

# Kategoriyi seçmek için dropdown menüsü
categories = [
    "Geräte & Zubehör",
    "Hilfe bei Störungen",
    "Internet & Telefonie",
    "MagentaEINS",
    "Mobilfunk",
    "TV",
    "Vertrag & Rechnung",
    "Apps & Dienste",
    "Others"
]

# Streamlit arayüzü
st.title("Soru-Cevap API")

selected_category = st.selectbox("Kategori Seçin:", categories)
question = st.text_input("Soru:", "")

if st.button("Cevapla"):
    if selected_category and question:
        # QA zincirini başlat
        qa_chain = initialize_qa_chain(selected_category)

        # Soruyu yanıtla
        answer = qa_chain.run(question)
        st.write("Cevap:", answer)
    else:
        st.warning("Lütfen kategori ve soru girin.")

2024-08-29 19:44:45.572 
  command:

    streamlit run /Users/taha/Desktop/rag/venv/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-08-29 19:44:45.572 Session state does not function when running a script without `streamlit run`
